This notebook contains code for converting epidemiological priors into the form required by our model. 

This involves jumps between different parameterisations, reproducing the method of feretti et al to compute the prior over the `GI_SD`, and summing delay distributions to form a global delay dist. 

In [ ]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm

In [ ]:
from epimodel.pymc3_models.epi_params import bootstrapped_negbinom_values, ci_to_mean_sd

import numpy as np
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline

import scipy.stats as stats

# reproduce feretti et al

This is required for the prior over the GI standard deviation. 

In [ ]:
# ci data from running feretti code, using the fitted gamma distribution
ci_data = np.loadtxt('ci_data.csv', delimiter=',', skiprows=1, usecols=[1, 2, 4])

In [ ]:
means = np.exp(ci_data[:, 0]) / np.exp(ci_data[:, 1])
sigmas = np.exp(0.5*ci_data[:, 0]) / np.exp(ci_data[:, 1])

# CI constructed using the likelihood ratio method, as in feretti et al
nLogL = ci_data[:, 2]

In [ ]:
ci_to_mean_sd(2.11, np.array([np.min(sigmas[nLogL < 3.84]), np.max(sigmas[nLogL < 3.84])]))

# produce incubation period

Here, we have been given `sd_mean`, and `sd_sd`, where `sd` actually refers to $\sigma$ parameter in the lognorm distribution, which is **not** the standard deviation. e.g., see here: https://en.wikipedia.org/wiki/Log-normal_distribution. We have these values from Lauer et al. 

We need to find `mean_mean` and `mean_sd` of the log mean (i.e., the $\mu$ parameter), but Fonfria et al report the actual mean of the distribution. Therefore, we grid over possible values of `mean_mean`, `mean_sd`. 


For each pair (corresponding to a setting of the prior), we draw `nRVs` distributions from the prior, and find a distribution over the mean values. We choose the pair that has mean close to the reported mean in Fonfria et al (in the code, within tolerance), and we prefer solutions which have smaller `mean_sd`. 

This process converts the prior over the distribution mean into a prior over $\mu$. 

In [ ]:
ms = np.linspace(1.5, 1.55, 51)
sds = np.linspace(0.05, 0.1, 51)

nRVs = int(1e7)

noise_mean = np.random.normal(0, 1, size=nRVs)
noise_sd = np.random.normal(0, 1, size=nRVs)

min_sd = 100
best_sol = np.array([0., 0.])
tol = 0.01

sd_mean = 0.418
sd_sd = 0.0759

for mean_mean in tqdm(ms.tolist()):
    for mean_sd in sds:
        log_mean = mean_mean + noise_mean * mean_sd
        log_sd = sd_mean + noise_sd * sd_sd
        means = np.exp(log_mean + 0.5 * log_sd ** 2)
        m = np.mean(means)
        l = np.percentile(means, 2.5)
        u = np.percentile(means, 97.5)
        if np.abs(m - 5.06) < tol:
            if l < 5.06 and u > 5.7:
                if mean_sd < min_sd:
                    min_sd = mean_sd
                    best_sol[0] = mean_mean
                    best_sol[1] = mean_sd
print(f'Best solution: {best_sol}')

In [ ]:
# final 
incubation_period = {
                'mean_mean': 1.53,
                'mean_sd': 0.051,
                'sd_mean': 0.418,
                'sd_sd': 0.0759,
                'source': 'Lauer et al, doi.org/10.7326/M20-0504'
                          'Fonfria et al, https://www.medrxiv.org/content/medrxiv/early/2020/06/19/2020.06.17.20133587',
                'dist': 'lognorm',
                'notes': 'mean_mean, mean_sd chosen from Fonfria et al, after fitting using Lauer values.'
                         '(log) sd, sd_sd taken from Lauer et al.'
            }

# bootstrap for global delays

In [ ]:
symptom_reporting = {
    'mean_mean': 5.82,
    'mean_sd': 0.68,
    'disp_mean': 1.57,
    'disp_sd': 0.054,
    'source': 'Cereda et al, https://arxiv.org/ftp/arxiv/papers/2003/2003.09320.pdf, §3'
              'Fonfria et al, https://www.medrxiv.org/content/medrxiv/early/2020/06/19/2020.06.17.20133587',
    'dist': 'negbinom',
    'notes': 'mean from Fronfria et al. Dispersions from Cereda et al.'
}

symptom_deaths = {
    'mean_mean': 16.71,
    'mean_sd': 0.75,
    'sd_mean': 6.9,
    'sd_sd': 1.122,
    'source': 'Linton et al'
              'Fonfria et al, https://www.medrxiv.org/content/medrxiv/early/2020/06/19/2020.06.17.20133587',
    'dist': 'gamma',
    'notes': 'Mean from Fronfria et al. Sd from Linton et al'
}

In [ ]:
bootstrapped_negbinom_values([incubation_period, symptom_reporting])

In [ ]:
bootstrapped_negbinom_values([incubation_period, symptom_deaths], truncation=48)